# Window Exercises

Examples taken from [Spark: The definitive Guide](https://github.com/databricks/Spark-The-Definitive-Guide)

## Imports and Initializing Spark Session

In [27]:
# Imports

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [6]:
# Initializing SparkSession

spark = SparkSession.builder.appName("Basic Structured Operations")\
.master("local[*]").getOrCreate()

## Loading Data

In [20]:
## Load Data

df = spark.read.csv("../data/retail/", header=True, 
                    inferSchema=True).withColumn("InvoiceDate", F.to_date(F.col("InvoiceDate")))

In [21]:
# Show data

df.show(5)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6| 2010-12-06|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8| 2010-12-06|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2| 2010-12-06|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6| 2010-12-06|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25| 2010-12-06|     0.42|   15987.0|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 5 rows



## Window

In [40]:
# Define the window
window = Window.partitionBy("InvoiceDate", "CustomerId").orderBy(F.desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

# colum that gets the maximum Quantity over the window
max_purchase_quantity = F.max(F.col("Quantity")).over(window)

# show results
df.select("CustomerId", "InvoiceDate", "Quantity", 
          max_purchase_quantity.alias("MaxQuantityUpToDate"),
          F.rank().over(window).alias("rank"),
          F.dense_rank().over(window).alias("dense_rank")).show()

+----------+-----------+--------+-------------------+----+----------+
|CustomerId|InvoiceDate|Quantity|MaxQuantityUpToDate|rank|dense_rank|
+----------+-----------+--------+-------------------+----+----------+
|   14001.0| 2010-12-01|      20|                 20|   1|         1|
|   14001.0| 2010-12-01|      16|                 20|   2|         2|
|   14001.0| 2010-12-01|      16|                 20|   2|         2|
|   14001.0| 2010-12-01|      10|                 20|   4|         3|
|   14001.0| 2010-12-01|      10|                 20|   4|         3|
|   14001.0| 2010-12-01|       8|                 20|   6|         4|
|   14001.0| 2010-12-01|       6|                 20|   7|         5|
|   14001.0| 2010-12-01|       6|                 20|   7|         5|
|   14001.0| 2010-12-01|       2|                 20|   9|         6|
|   15838.0| 2010-12-07|     300|                300|   1|         1|
|   15838.0| 2010-12-07|     150|                300|   2|         2|
|   15838.0| 2010-12

## Grouping Sets, Roll Up and Cube

In [57]:
df_no_null = df.filter(F.col("CustomerId").isNotNull())
df_no_null.registerTempTable("df_no_null")

In [58]:
## Using SQL & GROUPING SETS

spark.sql("""
    SELECT CustomerId, InvoiceDate, MAX(Quantity) as MaxQuantity
    from df_no_null
    GROUP BY CustomerId, InvoiceDate
    GROUPING SETS ((CustomerId, InvoiceDate), (InvoiceDate))
    ORDER BY InvoiceDate, MaxQuantity DESC

""").show()

+----------+-----------+-----------+
|CustomerId|InvoiceDate|MaxQuantity|
+----------+-----------+-----------+
|      null| 2010-12-01|        600|
|   13694.0| 2010-12-01|        600|
|   16210.0| 2010-12-01|        480|
|   16029.0| 2010-12-01|        432|
|   13777.0| 2010-12-01|        384|
|   17511.0| 2010-12-01|        288|
|   17181.0| 2010-12-01|        144|
|   14045.0| 2010-12-01|        128|
|   14688.0| 2010-12-01|        120|
|   12433.0| 2010-12-01|         96|
|   12791.0| 2010-12-01|         96|
|   13408.0| 2010-12-01|         96|
|   13093.0| 2010-12-01|         96|
|   13748.0| 2010-12-01|         80|
|   15485.0| 2010-12-01|         80|
|   18144.0| 2010-12-01|         80|
|   13255.0| 2010-12-01|         72|
|   17025.0| 2010-12-01|         72|
|   17760.0| 2010-12-01|         72|
|   15291.0| 2010-12-01|         64|
+----------+-----------+-----------+
only showing top 20 rows



In [59]:
## Using rollup

df_no_null.rollup("InvoiceDate", "CustomerId")\
.agg(F.max("Quantity").alias("MaxQuantity"))\
.orderBy(F.col("InvoiceDate"), F.desc("MaxQuantity")).show()

+-----------+----------+-----------+
|InvoiceDate|CustomerId|MaxQuantity|
+-----------+----------+-----------+
|       null|      null|       2880|
| 2010-12-01|   13694.0|        600|
| 2010-12-01|      null|        600|
| 2010-12-01|   16210.0|        480|
| 2010-12-01|   16029.0|        432|
| 2010-12-01|   13777.0|        384|
| 2010-12-01|   17511.0|        288|
| 2010-12-01|   17181.0|        144|
| 2010-12-01|   14045.0|        128|
| 2010-12-01|   14688.0|        120|
| 2010-12-01|   13408.0|         96|
| 2010-12-01|   13093.0|         96|
| 2010-12-01|   12791.0|         96|
| 2010-12-01|   12433.0|         96|
| 2010-12-01|   13748.0|         80|
| 2010-12-01|   18144.0|         80|
| 2010-12-01|   15485.0|         80|
| 2010-12-01|   17025.0|         72|
| 2010-12-01|   17760.0|         72|
| 2010-12-01|   13255.0|         72|
+-----------+----------+-----------+
only showing top 20 rows



In [61]:
## Using Cube

df_no_null.cube("InvoiceDate", "CustomerId")\
.agg(F.max("Quantity").alias("MaxQuantity"))\
.orderBy(F.col("InvoiceDate"), F.desc("MaxQuantity")).show()

+-----------+----------+-----------+
|InvoiceDate|CustomerId|MaxQuantity|
+-----------+----------+-----------+
|       null|   16754.0|       2880|
|       null|      null|       2880|
|       null|   16029.0|       2400|
|       null|   15299.0|       1824|
|       null|   12875.0|       1728|
|       null|   12755.0|       1488|
|       null|   14156.0|       1440|
|       null|   17857.0|       1394|
|       null|   17306.0|       1296|
|       null|   18102.0|       1008|
|       null|   13848.0|       1000|
|       null|   13027.0|        960|
|       null|   16422.0|        720|
|       null|   17381.0|        600|
|       null|   13694.0|        600|
|       null|   17404.0|        576|
|       null|   17511.0|        576|
|       null|   16013.0|        500|
|       null|   16927.0|        480|
|       null|   15061.0|        480|
+-----------+----------+-----------+
only showing top 20 rows



## Pivot

In [65]:
df.select("InvoiceDate", "Country", "Quantity").show()

+-----------+--------------+--------+
|InvoiceDate|       Country|Quantity|
+-----------+--------------+--------+
| 2010-12-06|United Kingdom|       6|
| 2010-12-06|United Kingdom|       8|
| 2010-12-06|United Kingdom|       2|
| 2010-12-06|United Kingdom|       6|
| 2010-12-06|United Kingdom|      25|
| 2010-12-06|United Kingdom|      25|
| 2010-12-06|United Kingdom|       2|
| 2010-12-06|United Kingdom|       6|
| 2010-12-06|United Kingdom|       6|
| 2010-12-06|United Kingdom|       2|
| 2010-12-06|United Kingdom|       6|
| 2010-12-06|United Kingdom|       2|
| 2010-12-06|United Kingdom|     120|
| 2010-12-06|United Kingdom|      48|
| 2010-12-06|United Kingdom|      48|
| 2010-12-06|United Kingdom|      48|
| 2010-12-06|United Kingdom|      12|
| 2010-12-06|United Kingdom|      48|
| 2010-12-06|United Kingdom|      12|
| 2010-12-06|United Kingdom|      12|
+-----------+--------------+--------+
only showing top 20 rows



In [68]:
df_pivoted = df.groupBy("InvoiceDate").pivot("Country").agg(F.sum("Quantity"))

In [69]:
df_pivoted.printSchema()

root
 |-- InvoiceDate: date (nullable = true)
 |-- Australia: long (nullable = true)
 |-- Austria: long (nullable = true)
 |-- Bahrain: long (nullable = true)
 |-- Belgium: long (nullable = true)
 |-- Channel Islands: long (nullable = true)
 |-- Cyprus: long (nullable = true)
 |-- Denmark: long (nullable = true)
 |-- EIRE: long (nullable = true)
 |-- Finland: long (nullable = true)
 |-- France: long (nullable = true)
 |-- Germany: long (nullable = true)
 |-- Iceland: long (nullable = true)
 |-- Israel: long (nullable = true)
 |-- Italy: long (nullable = true)
 |-- Japan: long (nullable = true)
 |-- Lithuania: long (nullable = true)
 |-- Netherlands: long (nullable = true)
 |-- Norway: long (nullable = true)
 |-- Poland: long (nullable = true)
 |-- Portugal: long (nullable = true)
 |-- Spain: long (nullable = true)
 |-- Sweden: long (nullable = true)
 |-- Switzerland: long (nullable = true)
 |-- United Kingdom: long (nullable = true)



In [71]:
df_pivoted.select("InvoiceDate", "Spain", "Switzerland", "United Kingdom").show()

+-----------+-----+-----------+--------------+
|InvoiceDate|Spain|Switzerland|United Kingdom|
+-----------+-----+-----------+--------------+
| 2010-12-15| null|       null|         18211|
| 2010-12-01| null|       null|         23949|
| 2010-12-02| null|       null|         20873|
| 2010-12-06| null|       null|         20669|
| 2010-12-09|   67|       null|         14545|
| 2010-12-21| null|       null|          8100|
| 2010-12-17|  100|       null|          8824|
| 2010-12-07| null|       null|         23769|
| 2010-12-08| null|       null|         20086|
| 2010-12-16| null|       null|         28060|
| 2010-12-12| null|       null|          8156|
| 2010-12-10| null|       null|         18273|
| 2010-12-13|  300|       null|         17325|
| 2010-12-14| null|       null|         19233|
| 2010-12-19| null|       null|          3303|
| 2010-12-23| null|       null|          4618|
| 2010-12-03|  400|        110|         10439|
| 2010-12-22| null|       null|          2350|
| 2010-12-05|